In [1]:
### 加载数据
import pandas
import pickle
from pyecharts.charts import Bar
from pyecharts import options as opts

SAVE_DATA = False
dataset = pandas.read_csv('dataset.csv')
print(f'Total data: {dataset.shape[0]}')

/Users/peanutzhen/Desktop/projects/techHW/venv/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3169: DtypeWarning: Columns (29) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,
Total data: 800000


In [3]:
### 清洗数据

# 洗掉不必要的列
data = dataset.drop(
    [
        'Unnamed: 0',
        '提交答卷时间',
        '所用时间',
        '来源',
        '来源详情',
        '来自IP',
        '1、请问您的学校所在地区和类别：'
    ],
    axis=1
)

# 列名含义，接下来使用下标访问
COLS_NAME = data.columns.values.tolist()
data.columns = [idx for idx in range(len(COLS_NAME))]

# 地区有两个-3的异常值 清洗
data = data[data[91] != -3]

# 14题数据为1or2 因此全体-1 统一处理
data.iloc[:,53:58] = data.iloc[:,53:58] - 1

# 线上学习时间有错误值及空值 0~15
data = data[data[22].isin([x for x in range(16)])].astype(int)

# 输出最后行数
print(f'Data cleaning completed！Total:\n{data.shape[0]}')

Data cleaning completed！Total:
750841


In [4]:
# 导出数据csv
if SAVE_DATA:
    data.to_csv('./cooked.csv')
    with open('./COLS_NAME.dat', 'wb') as f:
        pickle.dump(COLS_NAME, f)

In [8]:
# 分析参加调查的学生的年级分布（输出柱状图）
res = [0 for _ in range(12)]
for _, grade in data[1].items():
    res[grade-1] = res[grade-1] + 1

bar = (
    Bar()
    .add_xaxis(["一年级", "二年级", "三年级", "四年级", "五年级", "六年级", "初一", "初二", "初三", "高一", "高二", "高三"])
    .add_yaxis("问卷人数", res)
    .set_global_opts(
        title_opts=opts.TitleOpts(title="学生的年级分布"),
        legend_opts=opts.LegendOpts(is_show=True)
    )
)
bar.render_notebook()